In [1]:
import sys
import os
import math
import moveit_commander
import moveit_msgs.msg
import rospy
import numpy as np
import trajectory_msgs.msg
from geometry_msgs.msg import PoseStamped
import tf
import actionlib
import subprocess

### ALL OF THESE IMPORTS MUST BE HANDLeD BY TAKESHI INSTALL

In [2]:
rospy.init_node('moveiteando')

In [3]:
listener = tf.TransformListener()
from  utils_notebooks import *

In [4]:
arm = moveit_commander.MoveGroupCommander('arm')
base=  moveit_commander.MoveGroupCommander('base')
gripper = moveit_commander.MoveGroupCommander('gripper')
head = moveit_commander.MoveGroupCommander('head')

In [5]:
def gaze_point_floor(x,y):
    
    
    import moveit_commander
    import moveit_msgs.msg

   
    ## FUNCTION that points robot gaze to specific coordinates on the floor (z=0) retalive to the 'map' frame
    
    arm_pose=arm.get_current_joint_values()
    arm_pose[0]=.1
    arm_pose[1]= -0.3
    arm.set_joint_value_target(arm_pose)
    arm.go()
    head_pose = head.get_current_joint_values()
    head_pose[0]=0.0
    head_pose[1]=0.0
    head.set_joint_value_target(head_pose)
    head.go()
    trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0)) #
    np.rad2deg(head_pose)
    e =tf.transformations.euler_from_quaternion(rot)
    rospy.loginfo('i am at',trans[:2],np.rad2deg(e)[2])
    rospy.loginfo('gaze goal',x,y)
    #tf.transformations.euler_from_quaternion(rot)


    x_rob,y_rob,th_rob= trans[0], trans[1] ,  e[2]


    D_x=x_rob-x
    D_y=y_rob-y
    D_th= np.arctan2(D_y,D_x)
    print('relative to robot',(D_x,D_y,np.rad2deg(D_th)))

    pan_correct= (- th_rob + D_th + np.pi) % (2*np.pi)

    if(pan_correct > np.pi):
        pan_correct=-2*np.pi+pan_correct
    if(pan_correct < -np.pi):
        pan_correct=2*np.pi+pan_correct

    if ((pan_correct) > .5 * np.pi):
        print ('Exorcist alert')
        pan_correct=.5*np.pi
    head_pose[0]=pan_correct
    tilt_correct=np.arctan2(trans[2],np.linalg.norm((D_x,D_y)))

    head_pose [1]=-tilt_correct
    
    
    
    head.set_joint_value_target(head_pose)
    head.go()



In [10]:
gaze_point_floor(0,1)

TypeError: not all arguments converted during string formatting

In [10]:
def delete_object(name):
    u"""Gazeboの物体を消す関数

    引数:
        name (str): 物体の名前

    """

    cmd = ['rosservice', 'call', 'gazebo/delete_model',
           '{model_name: ' + str(name) + '}']
    subprocess.call(cmd)

    



In [11]:
#####SPAWN OBJECTS
from gazebo_ros import gazebo_interface
from geometry_msgs.msg import Pose, Quaternion
def spawn_object(gazebo_name, name, x, y, z, yaw):
    rospy.loginfo('Spawn: {0}'.format(name))
    initial_pose = Pose()
    initial_pose.position.x = x
    initial_pose.position.y = y
    initial_pose.position.z = z
    roll = 0.0
    pitch = 0.0

    q = tf.transformations.quaternion_from_euler(roll, pitch, yaw)
    initial_pose.orientation = Quaternion(q[0], q[1], q[2], q[3])

    path_xml = _path_xml.replace('MODEL_NAME', name)

    with open(path_xml, "r") as f:
        model_xml = f.read()

    model_xml = model_xml.replace('PATH_TO_MODEL', _path_model)

    gazebo_interface.spawn_sdf_model_client(gazebo_name, model_xml, rospy.get_namespace(),
                                            initial_pose, "", "/gazebo")


In [12]:
## THIS PATH MUST BE SET BY USER ()

#_path_xml = "/home/oscar/Codes/ycb_ws/src/robot_object_views/robotobject/models/MODEL_NAME/model-1_4.sdf"
#_path_model = "/home/oscar/Codes/ycb_ws/src/robot_object_views/robotobject/models"   Tenshi Version


_path_model = "/home/oscar/Codes/catkin_mio_ws/src/tmc_wrs_gazebo_world/models"     #Catkin mio version 
_path_xml = _path_model+"/MODEL_NAME/model-1_4.sdf"  #Catkin mio version


model_database_template = """<sdf version="1.4">
  <world name="default">
    <include>
      <uri>model://MODEL_NAME</uri>
    </include>
  </world>
</sdf>"""

objs=os.listdir(_path_model)
print (objs)

['ycb_021_bleach_cleanser', 'ycb_065-e_cups', 'wrc_bin_black', 'ycb_057_racquetball', 'wrc_tray', 'ycb_043_phillips_screwdriver', 'wrc_bin_green', 'ycb_073-b_lego_duplo', 'ycb_009_gelatin_box', 'ycb_072-e_toy_airplane', 'ycb_004_sugar_box', 'ycb_073-d_lego_duplo', 'ycb_061_foam_brick', 'ycb_016_pear', 'ycb_065-a_cups', 'wrc_ground_plane', 'wrc_bookshelf', 'ycb_072-b_toy_airplane', 'trofast', 'ycb_026_sponge', 'ycb_012_strawberry', 'ycb_013_apple', 'ycb_051_large_clamp', 'ycb_022_windex_bottle', 'ycb_065-h_cups', 'ycb_025_mug', 'ycb_036_wood_block', 'ycb_056_tennis_ball', 'ycb_044_flat_screwdriver', 'ycb_031_spoon', 'ycb_072-c_toy_airplane', 'wrc_frame', 'ycb_010_potted_meat_can', 'ycb_073-f_lego_duplo', 'ycb_072-d_toy_airplane', 'ycb_053_mini_soccer_ball', 'ycb_033_spatula', 'ycb_040_large_marker', 'ycb_027_skillet', 'ycb_052_extra_large_clamp', 'wrc_container_b', 'ycb_073-a_lego_duplo', 'ycb_070-a_colored_wood_blocks', 'wrc_stair_like_drawer', 'ycb_032_knife', 'wrc_long_table', 'ycb_0

In [13]:
def gazebo_2_world(x,y):

    x_world= x+2.5
    y_world= -(y-1.5)
    return (x_world,y_world)

def world_2_gazebo(x_world,y_world):

    x= ( x_world - 2.5)
    y= (-y_world + 1.5) 
    return (x , y)

In [14]:
z= 0 ## QUERY given Z where does the gaze ponits to wrt map


(trans,rot) = listener.lookupTransform('/map', '/head_rgbd_sensor_rgb_frame', rospy.Time(0))
e = np.rad2deg(tf.transformations.euler_from_quaternion(rot))
D_map=np.abs(trans[2]-z)/np.tan(np.deg2rad(e[0]-(-90)))
print ('x,y',D_map*-1* np.sin(np.deg2rad(e[2])), D_map* np.cos(np.deg2rad(e[2]))  )



('x,y', -885.5136923024259, 665.508900141093)


In [15]:
## WATCH YOUR GAZEBO WINDOW.. DID SOMETHING DROP ON THE ROTATING TABLE IN THE TRAIN WORLD?
## play around, try to drop and object and get close to it
## You might need to use the navigation funcions

In [16]:
model_name= objs[-2]#"073-g_lego_duplo"
print (model_name)





x_gaz,y_gaz=world_2_gazebo(1,1)


spawn_object(model_name, model_name, x_gaz,  y_gaz, .21, 0)  ### ORigin in gazebo world is in (-2, 1 ) map coords
#print (x_gaz,y_gaz)

#spawn_object("073-g_lego_duplo", "073-g_lego_duplo", x_gaz, y_gaz, .51, 0)



ycb_073-c_lego_duplo
[INFO] [1616441833.297506, 271.220000]: Spawn: ycb_073-c_lego_duplo
[INFO] [1616441833.303117, 271.220000]: Waiting for service /gazebo/spawn_sdf_model
[INFO] [1616441833.320817, 271.223000]: Calling service /gazebo/spawn_sdf_model
[INFO] [1616441833.616304, 271.271000]: Spawn status: SpawnModel: Successfully spawned entity


In [17]:
#CLEAN YOUR MESS ;)

In [9]:
delete_object(model_name)

In [29]:
def gaze_point_floor(x,y):
    ## FUNCTION that points robot gaze to specific coordinates on the floor (z=0) retalive to the 'map' frame
    
    arm_pose=arm.get_current_joint_values()
    arm_pose[0]=.1
    arm_pose[1]= -0.3
    arm.set_joint_value_target(arm_pose)
    arm.go()
    head_pose = head.get_current_joint_values()
    head_pose[0]=0.0
    head_pose[1]=0.0
    head.set_joint_value_target(head_pose)
    head.go()
    trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0)) #
    np.rad2deg(head_pose)
    e =tf.transformations.euler_from_quaternion(rot)
    print('i am at',trans[:2],np.rad2deg(e)[2])
    print('gaze goal',x,y)
    #tf.transformations.euler_from_quaternion(rot)


    x_rob,y_rob,th_rob= trans[0], trans[1] ,  e[2]


    D_x=x_rob-x
    D_y=y_rob-y
    D_th= np.arctan2(D_y,D_x)
    print('relative to robot',(D_x,D_y,np.rad2deg(D_th)))

    pan_correct= (- th_rob + D_th + np.pi) % (2*np.pi)

    if(pan_correct > np.pi):
        pan_correct=-2*np.pi+pan_correct
    if(pan_correct < -np.pi):
        pan_correct=2*np.pi+pan_correct

    if ((pan_correct) > .5 * np.pi):
        print ('Exorcist alert')
        pan_correct=.5*np.pi
    head_pose[0]=pan_correct
    tilt_correct=np.arctan2(trans[2],np.linalg.norm((D_x,D_y)))

    head_pose [1]=-tilt_correct
    
    
    
    head.set_joint_value_target(head_pose)
    head.go()

In [30]:
## Check both  moveit rviz and gazebo
## Takeshi is staring at rotating table  ... at least it should be. 

#gaze_point_floor(*gazebo_2_world(-1.5,1))
gaze_point_floor(*(1,1))

('i am at', [1.501338639493674, 3.3728574214301617], -179.7828949980177)
('gaze goal', 1, 1)
('relative to robot', (0.5013386394936741, 2.3728574214301617, 78.0699674299942))


In [14]:
gaze_point_floor(0,0)

TypeError: not all arguments converted during string formatting

In [22]:
#gaze_point_floor(x_world,y_world)

In [34]:
gaze_point_floor(*gazebo_2_world(-0.3,0.4))

('i am at', [-0.06008907449928854, 0.020884727133158934], -1.183550858187017)
('gaze goal', 2.2, 1.1)
('relative to robot', (-2.2600890744992888, -1.0791152728668412, -154.47711963433932))


In [23]:
gazebo_2_world(-1,.4)

(1.5, 1.1)

In [24]:
gaze_point_floor(*gazebo_2_world(-1,.4))

('i am at', [-0.06028384601783287, 0.02097001654934502], -1.2197839232187548)
('gaze goal', 1.5, 1.1)
('relative to robot', (-1.5602838460178328, -1.0790299834506552, -145.33381402352768))


In [26]:
gaze_point_floor(*gazebo_2_world(.7,1.6))

('i am at', [-0.06029738023023276, 0.02094663427183119], -1.1469958578341954)
('gaze goal', 3.2, -0.10000000000000009)
('relative to robot', (-3.260297380230233, 0.12094663427183128, 177.87548353198557))


In [19]:
## Homework

#1GO to gazebo and chose a random  object on the floor
#2 Locate object properties ( pose) and use the functions gaze point floor and gazebo 2 world to make the robot look towards that object
#3 Can you modifiy gaze_point_floor so it can point to a specific z (height from floor) value?

In [22]:
delete_object(model_name)

In [23]:
spawn_object(model_name, model_name, x_gaz,  y_gaz, .11, 45)  ### ORigin in gazebo world is in (-2, 1 ) map coords

[INFO] [1615935238.152363, 216.115000]: Spawn: ycb_073-c_lego_duplo
[INFO] [1615935238.156339, 216.115000]: Waiting for service /gazebo/spawn_sdf_model
[INFO] [1615935238.175363, 216.117000]: Calling service /gazebo/spawn_sdf_model
[INFO] [1615935238.283711, 216.120000]: Spawn status: SpawnModel: Successfully spawned entity


In [25]:

os.getcwd()

'/home/oscar/Codes/takeshi_vision/notebooks'

In [28]:
isdir = os.path.isdir('imagesfromlistener')

In [29]:
isdir


False

In [71]:
arm_pose=arm.get_current_joint_values()
arm_pose[0]=.1
arm_pose[1]= -0.3
arm.set_joint_value_target(arm_pose)
arm.go()
head_pose = head.get_current_joint_values()
head_pose[0]=0.0
head_pose[1]=0.0
head.set_joint_value_target(head_pose)
head.go()
trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0)) #

In [72]:
trans ,rot
x,y,z=1,2,0.4

In [73]:
## e =tf.transformations.euler_from_quaternion(rot)
print('i am at',trans,np.rad2deg(e)[2])
print('gaze goal',x,y,z)


('i am at', [1.468537482397813, 3.4205817402608707, 1.0170595715313226], 179.01597974295393)
('gaze goal', 1, 2, 0.4)


In [74]:
#tf.transformations.euler_from_quaternion(rot)


x_rob,y_rob,z_rob,th_rob = trans[0], trans[1], trans[2] ,  e[2]


D_x=x_rob-x
D_y=y_rob-y
D_z=z_rob-z

D_th= np.arctan2(D_y,D_x)
print('relative to robot',(D_x,D_y,D_z,np.rad2deg(D_th)))

('relative to robot', (0.468537482397813, 1.4205817402608707, 0.6170595715313226, 71.74636686280536))


In [75]:
 pan_correct= (- th_rob + D_th + np.pi) % (2*np.pi)

if(pan_correct > np.pi):
    pan_correct=-2*np.pi+pan_correct
if(pan_correct < -np.pi):
    pan_correct=2*np.pi+pan_correct

if ((pan_correct) > .5 * np.pi):
    print ('Exorcist alert')
    pan_correct=.5*np.pi
head_pose[0]=pan_correct
tilt_correct=np.arctan2((D_z),np.linalg.norm((D_x,D_y)))

head_pose [1]=-tilt_correct



head.set_joint_value_target(head_pose)
head.go()

True

In [76]:
gaze_point_floor(1,3)

('i am at', [1.4676253458062793, 3.4213453654731962], 179.83227715029972)
('gaze goal', 1, 3)
('relative to robot', (0.46762534580627935, 0.42134536547319623, 42.019861453906614))


In [92]:
def gaze_point(x,y,z):
    
    
    
    head_pose = head.get_current_joint_values()
    head_pose[0]=0.0
    head_pose[1]=0.0
    head.set_joint_value_target(head_pose)
    head.go()
    
    trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0)) #
    
    arm_pose=arm.get_current_joint_values()
    arm_pose[0]=.1
    arm_pose[1]= -0.3
    arm.set_joint_value_target(arm_pose)
    arm.go()
    
    e =tf.transformations.euler_from_quaternion(rot)
    print('i am at',trans,np.rad2deg(e)[2])
    print('gaze goal',x,y,z)
    #tf.transformations.euler_from_quaternion(rot)


    x_rob,y_rob,th_rob= trans[0], trans[1] ,  e[2]


    D_x=x_rob-x
    D_y=y_rob-y
    D_z=z_rob-z

    D_th= np.arctan2(D_y,D_x)
    print('relative to robot',(D_x,D_y,np.rad2deg(D_th)))

    pan_correct= (- th_rob + D_th + np.pi) % (2*np.pi)

    if(pan_correct > np.pi):
        pan_correct=-2*np.pi+pan_correct
    if(pan_correct < -np.pi):
        pan_correct=2*np.pi+pan_correct

    if ((pan_correct) > .5 * np.pi):
        print ('Exorcist alert')
        pan_correct=.5*np.pi
    head_pose[0]=pan_correct
    tilt_correct=np.arctan2(D_z,np.linalg.norm((D_x,D_y)))

    head_pose [1]=-tilt_correct
    
    
    
    head.set_joint_value_target(head_pose)
    succ=head.go()
    return succ

In [94]:
gaze_point(0,1.9,0.7)

('i am at', [-0.04697878886744429, 1.0019770459615636, 1.0170712934360158], 43.72067789539977)
('gaze goal', 0, 1.9, 0.7)
('relative to robot', (-0.04697878886744429, -0.8980229540384363, -92.99461710400833))


True